# Web Scraping Assignment_3

In [1]:
#lets now import all the required libraries
import selenium    
import pandas as pd    
from selenium import webdriver   
import warnings   
warnings.filterwarnings("ignore")
import time

#importing required exception which needs to handle
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

#importing requests
import requests

#importing regex
import re

In [2]:
#lets first connect to the web driver
driver = webdriver.Chrome(r"C:\Web Driver\chromedriver.exe")

# Answer-1 and 2:-

In [12]:
#Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

#First get the webpage https://www.amazon.in
driver.get('https://www.amazon.in')

In [13]:
#lets take a user input for search the product.
user_input = input("Enter the product you want to search :")

Enter the product you want to search :guitar


In [14]:
#lets search product that taken as input from user.
search_product = driver.find_element_by_xpath("/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[2]/div[1]/input")
search_product.send_keys(user_input)

In [15]:
#Then click the search button.
search_button = driver.find_element_by_xpath("/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[3]/div/span/input")
search_button.click()

In [16]:
#fetching url to open each product
product_opening_url = []
for i in range(0,3):
    url = driver.find_elements_by_xpath('//a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]')
    
    for j in url:
        try:
            product_opening_url.append(j.get_attribute('href'))
        except NoSuchElementException:
            product_opening_url.append('-')
        
    next_button = driver.find_elements_by_xpath("//a[@class='s-pagination-item s-pagination-next s-pagination-button s-pagination-separator']")
    try:
        driver.get(next_button[1].get_attribute('href'))
    except:
        driver.get(next_button[0].get_attribute('href'))

In [17]:
#display length of product url.
len(product_opening_url)

177

In [18]:
#display product url.
product_opening_url

['https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A0046612372SWJAK5T57D&url=%2FKadence-Frontier-Acoustic-Equalizer-Strings%2Fdp%2FB01HRIHH9E%2Fref%3Dsr_1_1_sspa%3Fcrid%3DMIH6HILGQOB8%26keywords%3Dguitar%26qid%3D1653815493%26sprefix%3Dguitar%252Caps%252C696%26sr%3D8-1-spons%26psc%3D1&qualifier=1653815493&id=4106551234435586&widgetName=sp_atf',
 'https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A061107531TUGN5RHMV0N&url=%2FKadence-Frontier-Acoustic-Guitar-Strings%2Fdp%2FB01GDZ46AA%2Fref%3Dsr_1_2_sspa%3Fcrid%3DMIH6HILGQOB8%26keywords%3Dguitar%26qid%3D1653815493%26smid%3DAM04Z7MH4HSDD%26sprefix%3Dguitar%252Caps%252C696%26sr%3D8-2-spons%26psc%3D1%26smid%3DAM04Z7MH4HSDD&qualifier=1653815493&id=4106551234435586&widgetName=sp_atf',
 'https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A06072772I2LJ344EB41Y&url=%2FPhotron-Acoustic-Cutaway-PH38C-BK%2Fdp%2F

In [19]:
#creating empty list to store brand name, model name, price, return, expected delivery, availability.
product_brand_name = []
product_name = []
product_price = []
product_return = []
product_expected_delivery = []
product_availability = []
product_url = []

In [20]:
#so lets extract all the tags having the product brand name, model name, price, return, expected delivery, availability.
for i in product_opening_url:
    driver.get(i)
    #time.sleep(5)
    try:
        brand_name = driver.find_element_by_xpath('//tr[@class="a-spacing-small po-brand"]/td[2]/span[1]')
        product_brand_name.append(brand_name.text)
    except NoSuchElementException:
        product_brand_name.append('-')
        
    try:
        name = driver.find_element_by_xpath('//h1[@class="a-size-large a-spacing-none"]/span')
        product_name.append(name.text)
    except NoSuchElementException:
        product_name.append('-')
    
    try:
        #price = driver.find_element_by_xpath('//div[@class="a-section a-spacing-none aok-align-center"]/span[2]/span[1]')
        price = driver.find_element_by_xpath("//span[@class='a-price aok-align-center reinventPricePriceToPayMargin priceToPay']/span[2]" or "//span[@class='a-price a-text-price a-size-medium apexPriceToPay']/span[2]")
        #price = driver.find_element_by_xpath("//span[@class='a-price']/span[2]")
        product_price.append(price.text.replace('\n','').replace('00',''))
    except NoSuchElementException:
        product_price.append('-')
        
    try:
        return_exchange = driver.find_element_by_xpath('//*[@id="RETURNS_POLICY"]/span/div[2]/a')
        product_return.append(return_exchange.text)
    except NoSuchElementException:
        product_return.append('-')
        
    try:
        expected_delivery = driver.find_element_by_xpath('//div[@class="a-spacing-base"]/b')
        product_expected_delivery.append(expected_delivery.text)
    except NoSuchElementException:
        product_expected_delivery.append('-')

    try:
        availability = driver.find_element_by_xpath('//div[@class="a-section a-spacing-none }"]/span')
        product_availability.append(availability.text)
    except NoSuchElementException:
        product_availability.append('-')

In [21]:
#Finally create a dataframe of the scraped data.
products = pd.DataFrame({})
products['brand'] = product_brand_name
products['name'] = product_name
products['price'] = product_price
products['return'] = product_return
products['expected_delivery'] = product_expected_delivery
products['availability'] = product_availability
products['url'] = product_opening_url
products

,brand,name,price,return,expected_delivery,availability,url
0,Kadence,Kadence Frontier Jumbo Semi Acoustic Guitar Wi...,-,7 Days Replacement,"Friday, June 3",In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
1,Kadence,Kadence Frontier guitar with Online Guitar lea...,-,7 Days Replacement,"Thursday, June 2",,https://www.amazon.in/gp/slredirect/picassoRed...
2,Photron,"Photron Acoustic Guitar, 38 Inch Cutaway, PH38...","₹2,490",7 Days Replacement,"Wednesday, June 1",In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
3,Blueberry,"Blueberry 38C, 38"" Acoustic Guitar Kit with Ba...","₹2,695",7 Days Replacement,"Thursday, June 2",In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
4,Juârez,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...","₹2,095",7 Days Replacement,"Tuesday, May 31",In stock.,https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...,...,...,...
172,KHUSHI MUSICALS,KHUSHI MUSICALS Finger Sleeve Silicone Fingert...,-,7 Days Replacement,-,-,https://www.amazon.in/KHUSHI-MUSICALS-Finger-S...
173,Intern,INTERN Cutaway Design Acoustic Guitar Pack -Hu...,"₹2,589",7 Days Replacement,"Tuesday, May 31",In stock.,https://www.amazon.in/Acoustic-Humidity-resist...
174,ARCTIC,"ARCTIC Sky series 40"" Semi-Acoustic Guitar (wi...","₹6,478",7 Days Replacement,"Wednesday, June 1",Only 2 left in stock.,https://www.amazon.in/ARCTIC-Semi-Acoustic-Gui...
175,JUAREZ,JUAREZ Fiésta 41 Inch Acoustic Guitar Cutaway ...,"₹3,999",7 Days Replacement,"Tuesday, May 31",In stock.,https://www.amazon.in/JUAREZ-Acoustic-Rosewood...


In [22]:
#save dataframe into csv file.
products.to_csv('guitars.csv')

# Answer-3:-

In [23]:
#Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

#First get the https://images.google.com/
driver.get('https://images.google.com/')

In [24]:
#lets search the fruits keyword name.
search_keyword = driver.find_element_by_xpath("/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input")
search_keyword.send_keys('fruits')

In [25]:
#Then click on the search button.
search_button = driver.find_element_by_xpath("/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/button")
search_button.click()

In [26]:
#get the url of 10 fruits images and download it.
for _ in range(20):
    driver.execute_script("window.scrollBy(0,1000)")
    
images = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')

img_urls = []
img_data = []
for image in images:
    source = image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)

for i in range(len(img_urls)):
    if i > 10:
        break
    print("Downloading {0} of {1} images" .format(i,10))
    response = requests.get(img_urls[i])
    file = open(r"C:/Users/dipak/Downloads/images/fruits/"+str(i)+".jpg","wb")
    file.write(response.content)

In [27]:
#First get the https://images.google.com/
driver.get('https://images.google.com/')

In [28]:
#lets search the cars keyword name.
search_keyword = driver.find_element_by_xpath("/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input")
search_keyword.send_keys('cars')

In [29]:
#Then click on the search button.
search_button = driver.find_element_by_xpath("/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/button")
search_button.click()

In [30]:
#get the url of 10 cars images and download it.
for _ in range(20):
    driver.execute_script("window.scrollBy(0,1000)")
    
images = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')

img_urls = []
img_data = []
for image in images:
    source = image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)

for i in range(len(img_urls)):
    if i > 10:
        break
    print("Downloading {0} of {1} images" .format(i,10))
    response = requests.get(img_urls[i])
    file = open(r"C:/Users/dipak/Downloads/images/cars/"+str(i)+".jpg","wb")
    file.write(response.content)

In [31]:
#First get the https://images.google.com/
driver.get('https://images.google.com/')

In [32]:
#lets search the machine learning keyword name.
search_keyword = driver.find_element_by_xpath("/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input")
search_keyword.send_keys('machine learning')

In [33]:
#Then click on the search button.
search_button = driver.find_element_by_xpath("/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/button")
search_button.click()

In [34]:
#get the url of 10 machine learning images and download it.
for _ in range(20):
    driver.execute_script("window.scrollBy(0,1000)")
    
images = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')

img_urls = []
img_data = []
for image in images:
    source = image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)

for i in range(len(img_urls)):
    if i > 10:
        break
    print("Downloading {0} of {1} images" .format(i,10))
    response = requests.get(img_urls[i])
    file = open(r"C:/Users/dipak/Downloads/images/machine learning/"+str(i)+".jpg","wb")
    file.write(response.content)

In [35]:
#First get the https://images.google.com/
driver.get('https://images.google.com/')

In [36]:
#lets search the guitar keyword name.
search_keyword = driver.find_element_by_xpath("/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input")
search_keyword.send_keys('guitar')

In [37]:
#Then click on the search button.
search_button = driver.find_element_by_xpath("/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/button")
search_button.click()

In [38]:
#get the url of 10 guitar images and download it.
for _ in range(20):
    driver.execute_script("window.scrollBy(0,1000)")
    
images = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')

img_urls = []
img_data = []
for image in images:
    source = image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)

for i in range(len(img_urls)):
    if i > 10:
        break
    print("Downloading {0} of {1} images" .format(i,10))
    response = requests.get(img_urls[i])
    file = open(r"C:/Users/dipak/Downloads/images/guitar/"+str(i)+".jpg","wb")
    file.write(response.content)

In [39]:
#First get the https://images.google.com/
driver.get('https://images.google.com/')

In [40]:
#lets search the cakes keyword name.
search_keyword = driver.find_element_by_xpath("/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input")
search_keyword.send_keys('cakes')

In [41]:
#Then click on the search button.
search_button = driver.find_element_by_xpath("/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/button")
search_button.click()

In [42]:
#get the url of 10 cakes images and download it.
for _ in range(20):
    driver.execute_script("window.scrollBy(0,1000)")
    
images = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')

img_urls = []
img_data = []
for image in images:
    source = image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)

for i in range(len(img_urls)):
    if i > 10:
        break
    print("Downloading {0} of {1} images" .format(i,10))
    response = requests.get(img_urls[i])
    file = open(r"C:/Users/dipak/Downloads/images/cakes/"+str(i)+".jpg","wb")
    file.write(response.content)

# Answer-4:-

In [46]:
#Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

#Go to Flipkart webpage by url : https://www.flipkart.com/
driver.get('https://www.flipkart.com/')

In [47]:
#search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com
search_field_products = driver.find_element_by_class_name("_3704LK")
search_field_products.send_keys('oneplus nord smartphone')

search_button = driver.find_element_by_xpath("/html/body/div[1]/div/div[1]/div[1]/div[2]/div[2]/form/div/button")
search_button.click()

In [48]:
#fetching url to open each product.
product_opening_url = []
url = driver.find_elements_by_xpath('//a[@class="_1fQZEK"]')
for j in url:
    try:
        product_opening_url.append(j.get_attribute('href'))
    except NoSuchElementException:
        product_opening_url.append('-')

In [49]:
#display length of product url.
len(product_opening_url)

24

In [50]:
#creating empty list to store product brands, description and price.
product_brand_name = []
product_name = []
product_colour = []
product_RAM = []
product_storage = []
product_primarycamera = []
product_secondarycamera = []
product_displaysize = []
product_batterycapacity = []
product_price = []
product_url = []

In [51]:
for i in product_opening_url:
    driver.get(i)
    #time.sleep(5)
    try:
        brand_name = driver.find_element_by_xpath('//span[@class="B_NuCI"]')
        product_brand_name.append(brand_name.text[0:7])
    except NoSuchElementException:
        product_brand_name.append('-')
        
    try:
        name = driver.find_element_by_xpath('//span[@class="B_NuCI"]')
        product_name.append(re.sub(r'\([^)]*\)','',name.text))
    except NoSuchElementException:
        product_name.append('-')
        
    try:
        colour = driver.find_element_by_xpath('//table[@class="_14cfVK"]/tbody/tr[4]/td[2]/ul/li')
        product_colour.append(colour.text)
    except NoSuchElementException:
        product_colour.append('-')
        
    try:
        ram = driver.find_element_by_xpath('//div[@class="_2418kt"]/ul/li[1]')
        product_RAM.append(ram.text.split('|')[0])
    except NoSuchElementException:
        product_RAM.append('-')
    
    try:
        rom = driver.find_element_by_xpath('//div[@class="_2418kt"]/ul/li[1]')
        product_storage.append(rom.text.split('|')[1])
    except NoSuchElementException:
        product_storage.append('-')
        
    try:
        primary_camera = driver.find_element_by_xpath('//div[@class="_2418kt"]/ul/li[3]')
        product_primarycamera.append(primary_camera.text.replace('Rear Camera',''))
    except NoSuchElementException:
        product_primarycamera.append('-')
    
    try:
        secondary_camera = 'not found'
        product_secondarycamera.append(secondary_camera)
    except NoSuchElementException:
        product_secondarycamera.append('-')
    
    try:
        display_size = driver.find_element_by_xpath('//div[@class="_2418kt"]/ul/li[2]')
        product_displaysize.append(display_size.text.replace('Display',''))
    except NoSuchElementException:
        product_displaysize.append('-')  
    
    try:
        battery_capacity = driver.find_element_by_xpath('//div[@class="_2418kt"]/ul/li[4]')
        product_batterycapacity.append(battery_capacity.text.replace('Battery',''))
    except NoSuchElementException:
        product_batterycapacity.append('-')  
        
    try:
        price = driver.find_element_by_xpath('//div[@class="_30jeq3 _16Jk6d"]')
        product_price.append(price.text)
    except NoSuchElementException:
        product_price.append('-') 

In [52]:
#Finally create a dataframe of the scraped data.
products = pd.DataFrame({})
products['brand'] = product_brand_name
products['name'] = product_name
products['colour'] = product_colour
products['ram'] = product_RAM
products['rom'] = product_storage
products['primary_camera'] = product_primarycamera
products['secondary_camera'] = product_secondarycamera
products['display_size'] = product_displaysize
products['battery_capacity'] = product_batterycapacity
products['price'] = product_price
products['url'] = product_opening_url
products

,brand,name,colour,ram,rom,primary_camera,secondary_camera,display_size,battery_capacity,price,url
0,OnePlus,OnePlus Nord 2 5G,Gray Sierra,8 GB RAM,128 GB ROM,50MP,not found,16.33 cm (6.43 inch),4500 mAh,"₹28,743",https://www.flipkart.com/oneplus-nord-2-5g-gra...
1,OnePlus,OnePlus Nord CE 2 Lite 5G,Black Dusk,6 GB RAM,128 GB ROM,64MP,not found,16.74 cm (6.59 inch),5000 mAh,"₹19,990",https://www.flipkart.com/oneplus-nord-ce-2-lit...
2,OnePlus,OnePlus Nord CE 2 Lite 5G,Black Dusk,8 GB RAM,128 GB ROM,64MP,not found,16.74 cm (6.59 inch),5000 mAh,"₹21,999",https://www.flipkart.com/oneplus-nord-ce-2-lit...
3,OnePlus,OnePlus Nord 2 5G,Green Wood,12 GB RAM,256 GB ROM,50MP,not found,16.33 cm (6.43 inch),4500 mAh,"₹34,999",https://www.flipkart.com/oneplus-nord-2-5g-gre...
4,-,-,-,-,-,-,not found,-,-,-,https://www.flipkart.com/oneplus-nord-ce-2-lit...
5,OnePlus,OnePlus Nord 2 5G,Blue Haze,8 GB RAM,128 GB ROM,50MP,not found,16.33 cm (6.43 inch),4500 mAh,"₹29,799",https://www.flipkart.com/oneplus-nord-2-5g-blu...
6,OnePlus,OnePlus Nord 2 5G,PAC-MAN,12 GB RAM,256 GB ROM,50MP,not found,16.33 cm (6.43 inch),4500 mAh,"₹33,999",https://www.flipkart.com/oneplus-nord-2-5g-pac...
7,OnePlus,OnePlus Nord CE 2 5G,Bahama Blue,6 GB RAM,128 GB ROM,64MP,not found,16.33 cm (6.43 inch),4500 mAh,"₹23,999",https://www.flipkart.com/oneplus-nord-ce-2-5g-...
8,OnePlus,OnePlus Nord CE 2 5G,Gray Mirror,6 GB RAM,128 GB ROM,64MP,not found,16.33 cm (6.43 inch),4500 mAh,"₹23,999",https://www.flipkart.com/oneplus-nord-ce-2-5g-...
9,OnePlus,OnePlus Nord CE 2 5G,Gray Mirror,8 GB RAM,128 GB ROM,64MP,not found,16.33 cm (6.43 inch),4500 mAh,"₹24,999",https://www.flipkart.com/oneplus-nord-ce-2-5g-...


In [65]:
#save dataframe into csv file.
products.to_csv('smartphones.csv')

# Answer-5:-

In [53]:
#Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

#First get the https://images.google.com/
driver.get('https://maps.google.com/')

In [54]:
#Enter location on search bar.
search_location = driver.find_element_by_class_name("tactile-searchbox-input")
search_location.send_keys('Rajkot Gujarat')

In [55]:
#Then click the search button.
search_button = driver.find_element_by_xpath("/html/body/div[3]/div[9]/div[3]/div[1]/div[1]/div[1]/div[2]/div[1]/button")
search_button.click()

In [56]:
#extraxt current url of given location.
try:
    url_string = driver.current_url
    print("URL Extracted:",url_string)
except NoSuchElementException:
    print("not found")

URL Extracted: https://www.google.com/maps/place/Rajkot,+Gujarat/@22.2734719,70.7512559,12z/data=!3m1!4b1!4m5!3m4!1s0x3959c98ac71cdf0f:0x76dd15cfbe93ad3b!8m2!3d22.3038945!4d70.8021599


In [57]:
#then extract latitue and longitude from given url string.
lat_long = re.findall(r"(?<=@)(.+?)(?=,1)",url_string)
lat_long = (" ".join(lat_long))
print("latitude and longitude of rajkot city:-",lat_long)

latitude and longitude of rajkot city:- 22.2734719,70.7512559


# Answer-6

In [58]:
#Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) from trak.in.

#Go to track webpage by url : https://trak.in//
driver.get('https://trak.in/')

In [59]:
#then click on funding deals link button.
fundingdeals_link_btn = driver.find_element_by_xpath("/html/body/div[1]/header/div[2]/div/div/div/div/nav/ul/li[9]/a")
fundingdeals_link_btn.click()

In [60]:
#create empty list to store sr.no, date, startupname, industry, sub-vertical, city, investor name, investment type and amount.
sr_no_list = []
date_list = []
startup_name_list = []
industry_list = []
sub_vertical_list = []
city_list = []
investor_name_list = []
investment_type_list = []
amount_list = []

In [61]:
#so lets extract all the tags having the sr.no, date, startupname, industry, sub-vertical, city, investor name, investment type and amount.

#extract sr.no tags
jan_sr_no_tags = driver.find_elements_by_xpath('/html/body/div[1]/main/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/div[5]/div[3]/div[2]/table/tbody/tr/td[1]')
feb_sr_no_tags = driver.find_elements_by_xpath('/html/body/div[1]/main/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/div[4]/div[3]/div[2]/table/tbody/tr/td[1]')
mar_sr_no_tags = driver.find_elements_by_xpath('/html/body/div[1]/main/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/div[3]/div[3]/div[2]/table/tbody/tr/td[1]')

for i in jan_sr_no_tags:
    jan_sr_no = i.text
    sr_no_list.append(jan_sr_no)
    
for j in feb_sr_no_tags:
    feb_sr_no = j.text
    sr_no_list.append(feb_sr_no)
    
for k in mar_sr_no_tags:
    mar_sr_no = k.text
    sr_no_list.append(mar_sr_no)
    
#extract date tags
jan_date_tags = driver.find_elements_by_xpath('/html/body/div[1]/main/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/div[5]/div[3]/div[2]/table/tbody/tr/td[2]')
feb_date_tags = driver.find_elements_by_xpath('/html/body/div[1]/main/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/div[4]/div[3]/div[2]/table/tbody/tr/td[2]')
mar_date_tags = driver.find_elements_by_xpath('/html/body/div[1]/main/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/div[3]/div[3]/div[2]/table/tbody/tr/td[2]')

for i in jan_date_tags:
    jan_date = i.text
    date_list.append(jan_date)
    
for j in feb_date_tags:
    feb_date = j.text
    date_list.append(feb_date)
    
for k in mar_date_tags:
    mar_date = k.text
    date_list.append(mar_date)
    
#extract startup tags
jan_startupname_tags = driver.find_elements_by_xpath('/html/body/div[1]/main/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/div[5]/div[3]/div[2]/table/tbody/tr/td[3]')
feb_startupname_tags = driver.find_elements_by_xpath('/html/body/div[1]/main/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/div[4]/div[3]/div[2]/table/tbody/tr/td[3]')
mar_startupname_tags = driver.find_elements_by_xpath('/html/body/div[1]/main/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/div[3]/div[3]/div[2]/table/tbody/tr/td[3]')

for i in jan_startupname_tags:
    jan_startupname = i.text
    startup_name_list.append(jan_startupname)
    
for j in feb_startupname_tags:
    feb_startupname = j.text
    startup_name_list.append(feb_startupname)
    
for k in mar_startupname_tags:
    mar_startupname = k.text
    startup_name_list.append(mar_startupname)

#extract industry tags
jan_industry_tags = driver.find_elements_by_xpath('/html/body/div[1]/main/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/div[5]/div[3]/div[2]/table/tbody/tr/td[4]')
feb_industry_tags = driver.find_elements_by_xpath('/html/body/div[1]/main/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/div[4]/div[3]/div[2]/table/tbody/tr/td[4]')
mar_industry_tags = driver.find_elements_by_xpath('/html/body/div[1]/main/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/div[3]/div[3]/div[2]/table/tbody/tr/td[4]')

for i in jan_industry_tags:
    jan_industry = i.text
    industry_list.append(jan_industry)
    
for j in feb_industry_tags:
    feb_industry = j.text
    industry_list.append(feb_industry)
    
for k in mar_industry_tags:
    mar_industry = k.text
    industry_list.append(mar_industry)
    
#extract sub-vartical tags
jan_subvartical_tags = driver.find_elements_by_xpath('/html/body/div[1]/main/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/div[5]/div[3]/div[2]/table/tbody/tr/td[5]')
feb_subvartical_tags = driver.find_elements_by_xpath('/html/body/div[1]/main/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/div[4]/div[3]/div[2]/table/tbody/tr/td[5]')
mar_subvartical_tags = driver.find_elements_by_xpath('/html/body/div[1]/main/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/div[3]/div[3]/div[2]/table/tbody/tr/td[5]')

for i in jan_subvartical_tags:
    jan_subvertical = i.text
    sub_vertical_list.append(jan_subvertical)
    
for j in feb_subvartical_tags:
    feb_subvertical = j.text
    sub_vertical_list.append(feb_subvertical)
    
for k in mar_subvartical_tags:
    mar_subvertical = k.text
    sub_vertical_list.append(mar_subvertical)

#extract city tags
jan_city_tags = driver.find_elements_by_xpath('/html/body/div[1]/main/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/div[5]/div[3]/div[2]/table/tbody/tr/td[6]')
feb_city_tags = driver.find_elements_by_xpath('/html/body/div[1]/main/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/div[4]/div[3]/div[2]/table/tbody/tr/td[6]')
mar_city_tags = driver.find_elements_by_xpath('/html/body/div[1]/main/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/div[3]/div[3]/div[2]/table/tbody/tr/td[6]')

for i in jan_city_tags:
    jan_city = i.text
    city_list.append(jan_city)
    
for j in feb_city_tags:
    feb_city = j.text
    city_list.append(feb_city)
    
for k in mar_city_tags:
    mar_city = k.text
    city_list.append(mar_city)

#extract investor name tags
jan_investorname_tags = driver.find_elements_by_xpath('/html/body/div[1]/main/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/div[5]/div[3]/div[2]/table/tbody/tr/td[7]')
feb_investorname_tags = driver.find_elements_by_xpath('/html/body/div[1]/main/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/div[4]/div[3]/div[2]/table/tbody/tr/td[7]')
mar_investorname_tags = driver.find_elements_by_xpath('/html/body/div[1]/main/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/div[3]/div[3]/div[2]/table/tbody/tr/td[7]')

for i in jan_investorname_tags:
    jan_investorname = i.text
    investor_name_list.append(jan_investorname)
    
for j in feb_investorname_tags:
    feb_investorname = j.text
    investor_name_list.append(feb_investorname)
    
for k in mar_investorname_tags:
    mar_investorname = k.text
    investor_name_list.append(mar_investorname)

#extract investment type tags
jan_investmenttype_tags = driver.find_elements_by_xpath('/html/body/div[1]/main/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/div[5]/div[3]/div[2]/table/tbody/tr/td[8]')
feb_investmenttype_tags = driver.find_elements_by_xpath('/html/body/div[1]/main/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/div[4]/div[3]/div[2]/table/tbody/tr/td[8]')
mar_investmenttype_tags = driver.find_elements_by_xpath('/html/body/div[1]/main/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/div[3]/div[3]/div[2]/table/tbody/tr/td[8]')

for i in jan_investmenttype_tags:
    jan_investmenttype = i.text
    investment_type_list.append(jan_investmenttype)
    
for j in feb_investmenttype_tags:
    feb_investmenttype = j.text
    investment_type_list.append(feb_investmenttype)
    
for k in mar_investmenttype_tags:
    mar_investmenttype = k.text
    investment_type_list.append(mar_investmenttype)

#extract amount tags
jan_amount_tags = driver.find_elements_by_xpath('/html/body/div[1]/main/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/div[5]/div[3]/div[2]/table/tbody/tr/td[9]')
feb_amount_tags = driver.find_elements_by_xpath('/html/body/div[1]/main/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/div[4]/div[3]/div[2]/table/tbody/tr/td[9]')
mar_amount_tags = driver.find_elements_by_xpath('/html/body/div[1]/main/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/div[3]/div[3]/div[2]/table/tbody/tr/td[9]')

for i in jan_amount_tags:
    jan_amount = i.text
    amount_list.append(jan_amount)
    
for j in feb_amount_tags:
    feb_amount = j.text
    amount_list.append(feb_amount)
    
for k in mar_amount_tags:
    mar_amount = k.text
    amount_list.append(mar_amount)

In [62]:
#display length of all the list.
print(len(sr_no_list),len(date_list),len(startup_name_list),len(industry_list),len(sub_vertical_list),len(city_list),len(investor_name_list),len(investment_type_list),len(amount_list))

24 24 24 24 24 24 24 24 24


In [64]:
#Finally create a dataframe of the scraped data.
data = pd.DataFrame({})
data['sr.no'] = sr_no_list
data['date'] = date_list
data['startup_name'] = startup_name_list
data['industry'] = industry_list
data['sub-vartical'] = sub_vertical_list
data['city'] = city_list
data['investor_name'] = investor_name_list
data['investment_type'] = investment_type_list
data['amount'] = amount_list
data

,sr.no,date,startup_name,industry,sub-vartical,city,investor_name,investment_type,amount
0,1,15/01/2021,Digit Insurance,Financial Services,Insurance Services,Bengaluru,"A91 Partners, Faering Capital, TVS Capital Funds",Venture,"1,80,00,000"
1,2,28/01/2021,Bombay Shaving Company,Consumer Goods Company,"Shave care, beard care, and skincare products",New Delhi,Reckitt Benckiser,Venture,"6,172,258.50"
2,3,19/01/2021,DeHaat,AgriTech Startup,online marketplace for farm products and services,Patna,Prosus Ventures,Series C,"30,000,000"
3,4,19/01/2021,Darwinbox,SaaS,HR Tech,Mumbai,Salesforce Ventures,Seed,"15,000,000"
4,5,18/01/2021,mfine,Health Tech Startup,AI-powered telemedicine mobile app,Bengaluru,Heritas Capital Management,Venture Round,"16,000,000"
5,6,18/01/2021,Udayy,EdTech,Online learning platform for kids in class 1-5,Gurgaon,Sequoia Capital,Seed Funding,"15,000,000"
6,7,11/01/2021,True Elements,Food Startup,Whole Food plant based Nashta,Pune,SIDBI Venture Capital,Series,"100,000,000"
7,8,13/01/2021,Saveo,B2B E-commerce,Pharmacies,Bengaluru,"Matrix Partners India, RTP Global, others",Seed,"4,000,000"
8,1,11/02/2021,Doubtnut,Edu Tech,E-Learning Platform,Gurgaon,"SIG Global, Sequoia Capital, WaterBridge Ventu...",Series B,"2,500,000"
9,2,22/02/2021,Zomato,Hospitality,Online Food Delivery Platform,Gurgaon,"Tiger Global, Kora",Venture,"250,000,000"


# Answer-7

In [66]:
#Write a program to scrap all the available details of best gaming laptops from digit.in

#Go to digit webpage by url : https://www.digit.in/
driver.get('https://www.digit.in/')

In [67]:
#then click on top10 link button.
top10_link_btn = driver.find_element_by_xpath("/html/body/div[1]/div/div[4]/ul/li[4]/a")
top10_link_btn.click()

In [68]:
#then click on laptop category.
laptop_category = driver.find_element_by_xpath("/html/body/div[3]/div/div/div[2]/div[5]/div[1]/div/button[2]")
laptop_category.click()

In [69]:
#then click on best gaming laptop in india sub-category.
best_gaming_laptop_subcategory = driver.find_element_by_xpath("/html/body/div[3]/div/div/div[2]/div[5]/div[3]/div[3]/a/div[2]/p")
best_gaming_laptop_subcategory.click()

In [70]:
#creating empty list to store product name, os, display, processor, memory, weight, dimension, graphics processor, desc.
product_name = []
product_os = []
product_display = []
product_processor = []
product_memory = []
product_weight = []
product_dimension = []
product_graphics_processor = []
product_desc = []

In [71]:
#lets extract all the tags.
name_tags = driver.find_elements_by_xpath('//div[@class="right-container"]/div/a/h3')
os_tags = driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr[3]/td[3]')
display_tags = driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr[4]/td[3]')
processor_tags = driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr[5]/td[3]')
memory_tags = driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr[6]/td[3]')
weight_tags = driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr[7]/td[3]')
dimension_tags = driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr[8]/td[3]')
graphics_tags = driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr[9]/td[3]')
description_tags = driver.find_elements_by_xpath('//div[@class="Section-center"]/p[1]')

In [72]:
#extract tags text and append into list.
for i in name_tags:
    try:
        name = i.text
        product_name.append(name)
    except NoSuchElementException:
        product_name.append('-')
        
for j in os_tags:        
    try:
        os = j.text
        product_os.append(os)
    except NoSuchElementException:
        product_os.append('-')
        
for k in display_tags:        
    try:
        display = k.text
        product_display.append(display)
    except NoSuchElementException:
        product_display.append('-')
        
for l in processor_tags:        
    try:
        processor = l.text
        product_processor.append(processor)
    except NoSuchElementException:
        product_processor.append('-')
        
for m in memory_tags:        
    try:
        memory = m.text
        product_memory.append(memory)
    except NoSuchElementException:
        product_memory.append('-')
        
for n in weight_tags:        
    try:
        weight = n.text
        product_weight.append(weight)
    except NoSuchElementException:
        product_weight.append('-')
        
for o in dimension_tags:        
    try:
        dimension = o.text
        product_dimension.append(dimension)
    except NoSuchElementException:
        product_dimension.append('-')
        
for p in graphics_tags:        
    try:
        graphics = p.text
        product_graphics_processor.append(graphics)
    except NoSuchElementException:
        product_graphics_processor.append('-')
        
for q in description_tags:        
    try:
        description = q.text
        product_desc.append(description)
    except NoSuchElementException:
        product_desc.append('-')

In [73]:
#Finally create a dataframe of the scraped data.
products = pd.DataFrame({})
products['name'] = product_name
products['os'] = product_os
products['display'] = product_display
products['processor'] = product_processor
products['memory'] = product_memory
products['weight'] = product_weight
products['dimension'] = product_dimension
products['graphics'] = product_graphics_processor
products['description'] = product_desc
products

,name,os,display,processor,memory,weight,dimension,graphics,description
0,MSI RAIDER GE76,Windows 11 Home,"17"" (3840 x 2160)",12th Gen Intel Core i9-12900HK | 5 GHz,2 TB SSD/16 GBGB DDR5,2.9,397 x 284 x 26,NVIDIA GeForce RTX 3080Ti,Possibly the most high-end gaming laptop anyon...
1,ASUS ROG STRIX SCAR 15,Windows 11 Home,"15.6"" (2560 x 1440)",12th Gen Intel Core i9-12900H | 2.5 GHz,2 TB SSD/32 GBGB DDR5,2.3,259 x 354 x 27,NVIDIA GeForce RTX 3070 Ti,The second high-end gaming laptop on the list ...
2,ACER NITRO 5,Windows 10,"15.6"" (1920 x 1080)",AMD Ryzen 9 Octa Core | 2.4 GHz,1 TB HDD/16 GBGB DDR4,2.4,363.4 x 255 x 23.9,NVIDIA GeForce RTX 3070,Possibly the best value-for-money gaming lapto...
3,MSI STEALTH 15M,Windows 10,"15.6"" (1920 x 1080)",Intel Core i7 11th Gen - 11375H | NA,1 TB SSD/16 GBGB DDR4,1.7,358.3 x 248 x 16.15,NVIDIA GeForce RTX 3060,If you’re looking for a powerful gaming laptop...
4,ASUS ROG STRIX SCAR 15,Windows 10,"15.6"" (2560 x 1440)",AMD Ryzen 9 Octa Core - 5900HX | 3.3 GHz,2 TB SSD/32 GBGB DDR4,2.30,354 x 259 x 22.6,NVIDIA GeForce RTX 3080,If you want possibly the best performance poss...
5,ASUS ROG STRIX SCAR 15,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 9 5900HX | 3.3 GHz,1 TB SSD/16 GBGB DDR4,2.30,35.4 x 25.9 x 2.26,NVIDIA® GeForce RTX™ 3070,When the ASUS ROG Strix Scar 15 ended up on ou...
6,ASUS ZEPHYRUS G14,Windows 10 Home,"14"" (1920 x 1080)",AMD 3rd Gen Ryzen 9 | 3.3 GHz,1 TB SSD/16 GBGB DDR4,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060,The Asus Zephyrus G14 is a first-of-its-kind g...
7,HP OMEN 16,Windows 11 Home,"16.1"" (1920 x 1080)",12th Gen Intel Core i7-12700H | 4.7 GHz,1 TB SSD/16 GBGB DDR4,2.32,36.92 x 24.8 x 2.3,NVIDIA GeForce RTX 3060,It is difficult to find a laptop that offers a...
8,ASUS ROG ZEPHYRUS DUO 15,Windows 10,"15.6"" (3840 x 1100)",Intel Core i7 10th Gen 10875H | NA,512 GB SSD/4 GBGB DDR4,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q,The machine is powered by an Intel Core i7-108...
9,ACER ASPIRE 7 GAMING LAPTOP,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 5-5500U hexa-core | NA,512 GB SSD/8 GBGB DDR4,2.15,2.29 x 36.3 x 25.4,NVIDIA® GeForce® GTX 1650,The Acer Aspire 7 gaming laptop is a very capa...


# Answer-8:-

In [3]:
#Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

#Go to forbes webpage by url : https://www.forbes.com/
driver.get('https://www.forbes.com/')

In [4]:
#then click on explore button.
explore_btn = driver.find_element_by_xpath("/html/body/div[1]/header/nav/div[1]/button[1]")
explore_btn.click()

In [5]:
#then hour the billionairs category for open sub category.
from selenium.webdriver.common.action_chains import ActionChains
element_to_hover_over = driver.find_element_by_xpath("/html/body/div[1]/header/nav/div[3]/ul/li[1]")
hover = ActionChains(driver).move_to_element(element_to_hover_over)
hover.perform()

#then click on world's billionairs sub category.
wbs = driver.find_element_by_xpath("/html/body/div[1]/header/nav/div[3]/ul/li[1]/div[2]/ul/li[2]/a")
wbs.click()

In [6]:
#creating empty list to store world's billionairs rank, name, networth, age, citizenship, source, industry.
billionairs_rank = []
billionairs_name = []
billionairs_networth = []
billionairs_age = []
billionairs_citizenship = []
billionairs_source = []
billionairs_industry = []

In [7]:
#you have to scrape the attributes.These are: 1. rank 2. name 3. networth 4. age 5. citizenship 6. source 7. industry
for i in range(0,13):
    rank_tags = driver.find_elements_by_xpath("//div[@class='table-row ']/div[1]")
    name_tags = driver.find_elements_by_xpath("//div[@class='table-row ']/div[2]")
    networth_tags = driver.find_elements_by_xpath("//div[@class='table-row ']/div[3]")
    age_tags = driver.find_elements_by_xpath("//div[@class='table-row ']/div[4]")
    citizenship_tags = driver.find_elements_by_xpath("//div[@class='table-row ']/div[5]")
    source_tags = driver.find_elements_by_xpath("//div[@class='table-row ']/div[6]")
    industry_tags = driver.find_elements_by_xpath("//div[@class='table-row ']/div[7]")
    
    for i in rank_tags:
        try:
            rank = i.text
            billionairs_rank.append(rank)
        except NoSuchElementException:
            billionairs_rank.append('-')
            
    for j in name_tags:
        try:
            name = j.text
            billionairs_name.append(name)
        except NoSuchElementException:
            billionairs_name.append('-')
    
    for k in networth_tags:
        try:
            networth = k.text
            billionairs_networth.append(networth)
        except NoSuchElementException:
            billionairs_networth.append('-')
            
    for l in age_tags:
        try:
            age = l.text
            billionairs_age.append(age)
        except NoSuchElementException:
            billionairs_age.append('-')
            
    for m in citizenship_tags:
        try:
            citizenship = m.text
            billionairs_citizenship.append(citizenship)
        except NoSuchElementException:
            billionairs_citizenship.append('-')
            
    for n in source_tags:
        try:
            source = n.text
            billionairs_source.append(source)
        except NoSuchElementException:
            billionairs_source.append('-')

    for o in industry_tags:
        try:
            industry = o.text
            billionairs_industry.append(industry)
        except NoSuchElementException:
            billionairs_industry.append('-')

    
    next_button = driver.find_element_by_xpath("/html/body/div[1]/div[1]/div/div/div[3]/div[2]/div[2]/div[2]/div[27]/div[2]/div[6]/div[1]/button[2]")
    next_button.click()      #locating web element of next button and then clicking on next button

In [8]:
#Finally create a dataframe of the scraped data.
billionairs = pd.DataFrame({})
billionairs['rank'] = billionairs_rank
billionairs['name'] = billionairs_name
billionairs['networth'] = billionairs_networth
billionairs['age'] = billionairs_age
billionairs['citizenship'] = billionairs_citizenship
billionairs['source'] = billionairs_source
billionairs['industry'] = billionairs_industry
billionairs

,rank,name,networth,age,citizenship,source,industry
0,1.,Elon Musk,$219 B,50,United States,"Tesla, SpaceX",Automotive
1,2.,Jeff Bezos,$171 B,58,United States,Amazon,Technology
2,3.,Bernard Arnault & family,$158 B,73,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$129 B,66,United States,Microsoft,Technology
4,5.,Warren Buffett,$118 B,91,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...
2583,2578.,Jorge Gallardo Ballart,$1 B,80,Spain,pharmaceuticals,Healthcare
2584,2578.,Nari Genomal,$1 B,82,Cyprus,apparel,Fashion & Retail
2585,2578.,Ramesh Genomal,$1 B,71,Cyprus,apparel,Fashion & Retail
2586,2578.,Sunder Genomal,$1 B,68,United Kingdom,garments,Fashion & Retail


# Answer-9:-

In [17]:
#Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

#Go to youtube webpage by url : https://www.youtube.com/
driver.get('https://www.youtube.com/')

In [18]:
#Enter video name on search bar.
search_video = driver.find_element_by_xpath("/html/body/ytd-app/div[1]/div/ytd-masthead/div[3]/div[2]/ytd-searchbox/form/div[1]/div[1]/input")
search_video.send_keys('srivalli song pushpa hindi')

In [19]:
#then click on search button.
search_button = driver.find_element_by_xpath('/html/body/ytd-app/div[1]/div/ytd-masthead/div[3]/div[2]/ytd-searchbox/button')
search_button.click()

In [20]:
#then click on video title link
title_link = driver.find_element_by_xpath('/html/body/ytd-app/div[1]/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-video-renderer[1]/div[1]/div/div[1]/div/h3/a')
title_link.click()

In [21]:
#creating empty list to store comments, comments upvote(like) and time when comment was posted of youtube video .
video_comments = []
video_comments_upvote = []
video_comments_post_time = []

In [22]:
#extract all the tags and append into list.
for _ in range(1000):
    driver.execute_script("window.scrollBy(0,10000)")
    
comment_tags = driver.find_elements_by_xpath('//ytd-expander[@class="style-scope ytd-comment-renderer"][1]')
comment_upvote_tags = driver.find_elements_by_xpath('//div[@class="style-scope ytd-comment-action-buttons-renderer"]/span[2]')
comment_post_time_tags = driver.find_elements_by_xpath('//yt-formatted-string[@class="published-time-text above-comment style-scope ytd-comment-renderer"]/a')
    
for i in comment_tags:
    try:
        comments = i.text
        video_comments.append(comments.replace('\n',''))
    except NoSuchElementException:
        video_comments.append('-')
        
    for i in range(len(video_comments)):
            if i > 50:
                break
            
for j in comment_upvote_tags:
    try:
        comment_upvote = j.text
        video_comments_upvote.append(comment_upvote)
    except NoSuchElementException:
        video_comments_upvote.append('-')
        
for j in range(len(video_comments_upvote)):
    if j > 500:
        break
            
for k in comment_post_time_tags:
    try:
        comment_post_time = k.text
        video_comments_post_time.append(comment_post_time)
    except NoSuchElementException:
        video_comments_post_time.append('-')
        
for k in range(len(video_comments_post_time)):
    if k > 500:
        break

In [24]:
#Finally create a dataframe of the scraped data.
contents = pd.DataFrame({})
contents['comments'] = video_comments
contents['comments_upvote'] = video_comments_upvote
contents['comments_post_time'] = video_comments_post_time
contents.head(500)

,comments,comments_upvote,comments_post_time
0,"Never ignore a person who loves you, cares for...",1K,5 months ago
1,I just love this song.,352,3 months ago
2,Is song mein ek nasha hai 2 mahine ho gaye lek...,4,2 months ago
3,Javed Ali's voice is so melodious and soothing...,139,2 months ago
4,The music composition is so good that it jels...,321,4 months ago (edited)
...,...,...,...
495,The best song 👌,3,1 month ago
496,What a great singer jawed Ali very smooth voic...,,2 months ago
497,"Javed Ali Sir voice is So Pure""❤""",39,5 months ago
498,मी हे गाणं दिवसातून खुप वेळा ऐकते खुप छान आहे ...,,3 months ago


# Answer-10:-

In [105]:
#Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

#Go to hostelworld webpage by url : https://www.hostelworld.com/
driver.get('https://www.hostelworld.com/')

In [106]:
#click on searchbar for search location then search location name on search bar..
click_search = driver.find_element_by_xpath("/html/body/div[3]/div/div/div[1]/div[1]/div/div[2]/div[4]/div/div[2]/div/div[1]/div")
click_search.click()

search_location = driver.find_element_by_xpath("/html/body/div[3]/div/div/div[1]/div[1]/div/div[2]/div[4]/div/div[2]/div/div[1]/div/div/div/input")
search_location.send_keys('london')

In [107]:
#then click on suggestion list of london city
click_location_suggestion = driver.find_element_by_xpath("/html/body/div[3]/div/div/div[1]/div[1]/div/div[2]/div[4]/div/div[2]/div/div[1]/div/div/ul/li[2]")
click_location_suggestion.click()

In [108]:
#then click on search button.
search_btn = driver.find_element_by_xpath('/html/body/div[3]/div/div/div[1]/div[1]/div/div[2]/div[4]/div/div[2]/div/div[5]/button')
search_btn.click()

In [109]:
#fetching url to open each hostel
hostel_opening_url = []
url = driver.find_elements_by_xpath('//h2[@class="title title-6"]/a')
for i in url:
    try:
        hostel_opening_url.append(i.get_attribute('href'))
    except NoSuchElementException:
        hostel_opening_url.append('-')

In [113]:
#fetching next page url to open each hostel
url = driver.find_elements_by_xpath('//h2[@class="title title-6"]/a')
for i in url:
    try:
        hostel_opening_url.append(i.get_attribute('href'))
    except NoSuchElementException:
        hostel_opening_url.append('-')

In [114]:
#display length of hostel link url.
len(hostel_opening_url)

47

In [115]:
#creating empty list to store hostel name, distance from city centre, ratings, total reviews, overall reviews, private and dorms from price, facilities and property description.
hostel_name_list = []
hostel_rating_list = []
hostel_total_review_list = []
hostel_overall_review_list = []
hostel_free_facilities_list = []
hostel_general_facilities_list = []
hostel_services_facilities_list = []
hostel_description_list = []
hostel_distance_from_city_centre_list = []
hostel_private_price_list = []
hostel_dorms_price_list = []

In [18]:
#extract all the tags and append into list.
for i in hostel_opening_url:
    driver.get(i)
    try:
        name = driver.find_element_by_xpath('//div[@class="title-2"]')
        hostel_name_list.append(name.text)
    except NoSuchElementException:
        hostel_name_list.append('-')
        
    try:
        ratings = driver.find_element_by_xpath('//div[@class="flex-20"]/div[1]/div[1]')
        hostel_rating_list.append(ratings.text)
    except NoSuchElementException:
        hostel_rating_list.append('-')
        
    try:
        total_review = driver.find_element_by_xpath('//div[@class="flex-20"]/div[1]/div[2]/div[2]')
        hostel_total_review_list.append(total_review.text)
    except NoSuchElementException:
        hostel_total_review_list.append('-')
        
    try:
        overall_review = driver.find_element_by_xpath('//div[@class="flex-20"]/div[1]/div[2]/div[1]/span')
        hostel_overall_review_list.append(overall_review.text)
    except NoSuchElementException:
        hostel_overall_review_list.append('-')
        
    try:
        free_facility = driver.find_element_by_xpath('//ul[@class="facility-sections"]/li[1]/ul')
        hostel_free_facilities_list.append(free_facility.text)
    except NoSuchElementException:
        hostel_free_facilities_list.append('-')
        
    try:
        general_facility = driver.find_element_by_xpath('//ul[@class="facility-sections"]/li[2]/ul')
        hostel_general_facilities_list.append(general_facility.text)
    except NoSuchElementException:
        hostel_general_facilities_list.append('-')
        
    try:
        services_facility = driver.find_element_by_xpath('//ul[@class="facility-sections"]/li[3]/ul')
        hostel_services_facilities_list.append(services_facility.text)
    except NoSuchElementException:
        hostel_services_facilities_list.append('-')
        
    try:
        description = driver.find_element_by_xpath('//div[@class="description-container"]/div/div[2]')
        hostel_description_list.append(description.text)
    except NoSuchElementException:
        hostel_description_list.append('-')

In [19]:
#Finally create a dataframe of the scraped data.
hostels = pd.DataFrame({})
hostels['name'] = hostel_name_list
hostels['ratings'] = hostel_rating_list
hostels['total_review'] = hostel_total_review_list
hostels['overall_review'] = hostel_overall_review_list
hostels['free_facility'] = hostel_free_facilities_list
hostels['general_facility'] = hostel_general_facilities_list
hostels['services_facility'] = hostel_services_facilities_list
hostels['description'] = hostel_description_list

hostels

,name,ratings,total_review,overall_review,free_facility,general_facility,services_facility,description
0,St Christopher's Village,8.0,11211 Total Reviews,Fabulous,Linen Included Free City Maps Free WiFi Free I...,Security Lockers Key Card Access Common Room E...,Internet access Laundry Facilities Towels for ...,Hostelworld visitors have said St Christopher'...
1,Selina Camden,9.1,25 Total Reviews,Superb,Linen Included Towels Included Free WiFi,Security Lockers Air Conditioning Flexible NRR,Laundry Facilities Towels for hire 24 Hour Rec...,"Among underground music venues, innovative mus..."
2,Generator London,7.7,6949 Total Reviews,Very Good,Linen Included Free City Maps Free WiFi Free I...,Wheelchair Friendly Security Lockers Key Card ...,Internet access Laundry Facilities Airport Tra...,Hostelworld guests have said Generator Hostel ...
3,Safestay London Elephant & Castle,7.2,4251 Total Reviews,Very Good,Linen Included Free City Maps Free WiFi,Security Lockers Key Card Access Common Room E...,Internet access Laundry Facilities Towels for ...,Safestay at Elephant and Castle is located in ...
4,London Backpackers,8.1,4242 Total Reviews,Fabulous,Free Breakfast Linen Included Free City Maps F...,Security Lockers Key Card Access Common Room A...,Internet access Laundry Facilities Towels for ...,IMPORTANT NOTICE: This hostel is for 18 - 35-y...
5,Barmy Badger Backpackers,9.2,1738 Total Reviews,Superb,Free Breakfast Linen Included Free City Maps F...,Security Lockers Common Room Bicycle Parking B...,Laundry Facilities Towels for hire Luggage Sto...,The Badger is unique for London in that we hav...
6,Safestay London Kensington Holland Park,6.7,1195 Total Reviews,Good,Linen Included Free WiFi,Security Lockers Key Card Access Breakfast Not...,Laundry Facilities Luggage Storage 24 Hour Rec...,Safestay Holland Park\n\nSafestay Holland Park...
7,Smart Hyde Park View Hostel,8.0,4549 Total Reviews,Fabulous,Linen Included Free City Maps Free WiFi,Security Lockers Key Card Access Common Room E...,Laundry Facilities Luggage Storage 24 Hour Rec...,"Smart Hyde Park View is a 'clean, pleasant pla..."
8,Wombat's City Hostel London,9.1,13615 Total Reviews,Superb,Linen Included Free City Maps Free WiFi Free I...,Wheelchair Friendly Security Lockers Key Card ...,Internet access Laundry Facilities Bicycle Hir...,A safe haven in the middle of the metropolis: ...
9,Queen Elizabeth Chelsea,6.9,3263 Total Reviews,Good,Linen Included Free City Maps Free WiFi Free I...,Security Lockers Common Room Breakfast Not Inc...,Internet access Luggage Storage 24 Hour Recept...,PLEASE NOTE WE ONLY ACCEPT GUESTS FROM THE AGE...
